In [ ]:

import os, time, pandas as pd,random
from datetime import datetime

# Output directory
output_dir = "sensor_data/csv"
os.makedirs(output_dir, exist_ok=True)

# Define the headers for the CSV file related to customer transactions
header = ['sensor_id', 'timestamp', 'temperature', 'humidity']


In [4]:

# Generate random temperature and humidity data
def generate_temperature_humidity_data(sensor_id=1):
    return {
        "sensor_id": sensor_id,
        "timestamp": datetime.now().isoformat(),
        "temperature": round(random.uniform(18, 30), 2),
        "humidity": round(random.uniform(40, 70), 2)
    }


In [ ]:
# Save data as CSV
def save_temperature_humidity_csv(filename, data):
    filepath = os.path.join(output_dir, filename)
    df = pd.DataFrame(data)
    df.to_csv(filepath, mode='w', index=False, header=True)  
    print(f"Saved Temperature & Humidity CSV: {filepath}")

In [7]:

# User input for number of times to run
num_iterations = int(input("Enter number of readings to generate: "))

temperature_humidity_sensor_data = []
# Generate and save sensor data
for i in range(num_iterations):
    temperature_humidity_data = generate_temperature_humidity_data()
    temperature_humidity_sensor_data.append(temperature_humidity_data)
    print(temperature_humidity_data)
    time.sleep(1)

save_temperature_humidity_csv("temperature_humidity.csv", temperature_humidity_sensor_data)


{'sensor_id': 1, 'timestamp': '2025-03-02T14:38:04.124482', 'temperature': 26.78, 'humidity': 51.39}
{'sensor_id': 1, 'timestamp': '2025-03-02T14:38:05.125005', 'temperature': 24.09, 'humidity': 64.74}
Saved Temperature & Humidity CSV: sensor_data/csv\temperature_humidity.csv
